In [ ]:
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

# PySpark Intro
<!-- requirement: small_data/employee -->
<!-- requirement: small_data/gutenberg -->
<!-- requirement: small_data/eeg -->

Like _Hadoop_, __Spark__ is a low-level system for distributed computation on a cluster.  It has two major advantages:
 
 - It can do in-memory caching between stages, which improves performance.  It also makes it suitable for classes of algorithms that would otherwise be too slow (*e.g.* iterative algorithms, like the training step in many machine learning algorithms).
 
 - It has a more flexible execution model and a more expressive API.  This allows you to easily write things beyond map and reduce.
 
There are also minor advantages: The default API is much nicer than writing raw MR code. The favored interface is via a language called Scala, but there is also good support for using Python.


### References


* The [Spark homepage](http://spark.apache.org) is your best starting point for high level programming guides, API docs, the blog, and in general most things you'll need.
* The [GitHub page](https://github.com/apache/spark) can be useful if you want to dig through source code.
* The [DataBricks Community Edition](https://community.cloud.databricks.com) is a free Spark notebook sandbox that also includes a bunch of useful tutorials.
* There's a GitBook called [Mastering Apache Spark 2.0](https://www.gitbook.com/book/jaceklaskowski/mastering-apache-spark/details).


## How does Spark relate to Hadoop?  


__Answer 1__: It's a replacement for it.  You can manage and run your own Spark cluster, independent of Hadoop.  You'll have to figure out the file system layer from scratch though.   (In this context, _Spark SQL_ is the replacement for _Hive_, i.e. for SQL-like operation.)

__Answer 2__: It's a complement to it.  You can run Spark on top of a Hadoop cluster, and still leverage HDFS and YARN -- then Spark is just replacing MapReduce.  (In this context, _Spark SQL_ can be used as a drop-in replacement for _Hive_.)


## The Spark API

### Nouns

The two main abstractions in Spark are:
1. Resilient distributed data sets (RDDs):
  This is like a (smartly) distributed list, which is partitioned across the nodes of the cluster and can be operated on in parallel.  The operations take the form of the usual functional list operations as shown above.  There are two types of operations: Transformations and Actions.
1. Shared variables used in parallel operations: When Spark runs a function in parallel across nodes, it ships a copy of each variable needed by that function to each node.  There are two types of shared variables:
 - Broadcast variables: These are used to store a value in memory across all nodes (i.e. their values are "broadcast" from the driver node)
 - Accumulator variables: These are variables which are only added to across nodes, for implementing counters and sums.
 

### Verbs

- **Transformations:** This creates a new RDD from an old one: for instance `map` or `flatMap`.  Transformations in Spark are __lazy__, which means that they do not compute the answer right away -- only when it is needed by something else.  Instead they represent the "idea" of the transformation applied to the base data set (e.g. for each chaining).  In particular, this means that intermediate results to computations do not necessarily get created and stored -- if the output of your map is going straight into a reduce, Spark should be clever enough to realize this and not actually store the output of the map.  Another consequence of this design is that the results of transformations are, by default, recomputed each time they are needed -- to store them one must explicitly call `cache`.
    
    [Transformations](https://spark.apache.org/docs/latest/programming-guide.html#transformations) typically *transform* the data and return another RDD.  Common examples include `map`, `filter`, `flatMap`, and also `join`, `cogroup`, `groupByKey`, `reduceByKey`, `countByKey`, and `sample`.

- **Actions:** These actually return a value as a result of a computation.  For instance, `reduce` is an action that combines all elements of an RDD using some function and then returns the final result.  (Note that `reduceByKey` actually returns an RDD.)
    
    [Actions](https://spark.apache.org/docs/latest/programming-guide.html#actions) typically either generate *small* outputs (e.g. `reduce`, `count`, `first`, `take`, `takeSample`, `foreach`, `collect`) or persist to disk (e.g. `saveAsTextFile`, `saveAsSequenceFile`, etc.).


**Questions:**
1. What's the difference between `map` and `foreach` in (non-Spark) Scala?  Why is `map` a transformation but `foreach` an action in Spark?
1. Why is `reduceByKey` a transformation but `reduce` an action?

## Word Count example

A common distributed-computing example is word count: count the number of times a word appears in a large corpus of text.  It is easily parallelizable, since counts can be made on different parts of the text and then added together to get the final count.

We start off by making a Spark **context**.  This object manages the resources available to a Spark job.

In [ ]:
from pyspark import SparkContext
sc = SparkContext("local[*]", "temp")

Spark will work on a distributed file system by default.  To tell it to deal with local files, we introduce the `localpath` convenience function.

In [ ]:
import os, time
def localpath(path):
    return 'file://' + os.path.join(os.path.abspath(os.path.curdir), path)

Using the `textFile` function, we create a RDD that contains all of the lines for all of the files in a directory.

In [ ]:
lines = sc.textFile(localpath('small_data/gutenberg/'))

The `count` action will tell us the total number of lines in these files.

In [ ]:
lines.count()

To do useful calculations, we will chain transformations together.  Here, a `flatMap` converts the RDD of lines to a RDD of words.  The `map` makes a tuple of `(word, 1)`.  The word acts as a key for the `reduceByKey` transformation, which sums up the values associated with each key, giving us the total count for each word.  Finally, we sort the RDD and write it out to a file.

In [ ]:
filename = "/tmp/output_gutenberg" + str(time.time())

lines.flatMap(lambda line: line.split(" ")) \
     .map(lambda word: (word.lower(), 1)) \
     .reduceByKey(lambda x, y: x + y) \
     .sortByKey() \
     .saveAsTextFile(localpath(filename))

In [ ]:
! head $filename/part-00000

Let's see what happened in each of these steps.  If we just looks at `lines`, all we see is that it is an RDD.

In [ ]:
lines

We don't see any of the lines themselves.  This is a result of Spark's lazy computation.  Only when an action requests some data is a computation actually run.  The `collect()` action will gather all of the elements in the RDD into a Python list.  We don't want all of these, so we'll use `take(5)` to gather five elements from the RDD.  Spark will do the minimal computation necessary to get those five elements; here we see that they come from the beginning of one of the books.

In [ ]:
lines.take(5)

The elements in the `lines` RDD are whole lines.  The `flatMap` transformation will create a new RDD containing the words.

In [ ]:
lines.flatMap(lambda line: line.split(" ")) \
     .take(5)

Note the difference from the basic `map` transformation, which would convert each line into a list of words.

In [ ]:
lines.map(lambda line: line.split(" ")) \
     .take(5)

Next, we will use a `map`, now to create those key-value pairs.  We're keying by the word, with the count as the value.  Each time we see a word, it's just a count of 1.

In [ ]:
lines.flatMap(lambda line: line.split(" ")) \
     .map(lambda word: (word.lower(), 1)) \
     .take(5)

Now `reduceByKey` will gather like keys together into a single entry, whose value is the result of running the specified reduction function, in this case summation, on all the values associated with that key.  This give us the counts per word.

In [ ]:
lines.flatMap(lambda line: line.split(" ")) \
     .map(lambda word: (word.lower(), 1)) \
     .reduceByKey(lambda x, y: x + y) \
     .take(5)

The `sortByKey` will alphabetize the RDD, for our convenience.

In [ ]:
lines.flatMap(lambda line: line.split(" ")) \
     .map(lambda word: (word.lower(), 1)) \
     .reduceByKey(lambda x, y: x + y) \
     .sortByKey() \
     .take(5)

**Exercise:** There are a lot of special symbols in the output.  How would you do word count only on pure characters?  Hint: in Python, you can test for regular expressions this way:

In [ ]:
import re
matches = re.search("^\\w+$", "abc")
matches.group(0)

### Example: a simple simulation for $\pi$

In [ ]:
import random
numSamples = 100000

def generate_point(_):
    x = random.random()
    y = random.random()
    if x * x + y * y < 1:
        return 1
    else:
        return 0

count = sc.parallelize(range(1, numSamples + 1)) \
    .map(generate_point) \
    .reduce(lambda x, y: x + y)
    
print("Pi is roughly " + str(4.0 * count / numSamples))

## ETL example

The first steps of any data science project usually involve ETL and exploratory analytics on a data set.
Here we'll acquaint ourselves both with the Spark shell and some Spark functions that we can use to perform such tasks.

#### The data set
From UC Irvine's [Machine Learning Archive](http://cml.ics.uci.edu/):
> "The first four lines are header information. Line 1 contains the subject identifier and indicates if the subject was an alcoholic (a) or control (c) subject by the fourth letter. Line 4 identifies the matching conditions: a single object shown (`S1 obj`), object 2 shown in a matching condition (`S2 match`), and object 2 shown in a non matching condition (`S2 nomatch`). Line 5 identifies the start of the data from sensor `FP1`. The four columns of data are: the trial number, sensor position, sample number (0-255), and sensor value (in micro volts)."

There are 16,452 rows in this file including the header. Here's a preview of the first 10 lines and last 10 lines:
```
# co2a0000364.rd
# 120 trials, 64 chans, 416 samples 368 post_stim samples
# 3.906000 msecs uV
# S1 obj , trial 0
# FP1 chan 0
0 FP1 0 -8.921
0 FP1 1 -8.433
0 FP1 2 -2.574
0 FP1 3 5.239
0 FP1 4 11.587
    ...
0 Y 246 24.150
0 Y 247 20.243
0 Y 248 11.454
0 Y 249 4.618
0 Y 250 3.153
0 Y 251 6.571
0 Y 252 12.431
0 Y 253 15.849
0 Y 254 16.337
0 Y 255 14.872
```

We need to filter out the header lines, which begin with a '#'.  We'll also make a simple class to represent each row.

In [ ]:
def isHeader(line):
    return "# " in line

class Record(object):
    def __init__(self, trial, posn, sample, reading):
        self.trial = trial
        self.posn = posn
        self.sample = sample
        self.reading = reading

    @classmethod
    def parse(cls, line):
        tokens = line.split()
        trial = int(tokens[0])
        posn = tokens[1]
        sample = int(tokens[2])
        reading = float(tokens[3])
        return cls(trial, posn, sample, reading)

data = sc.textFile(localpath("small_data/eeg/*")) \
    .filter(lambda x: not isHeader(x)) \
    .map(Record.parse)

There is an action for `RDD[Double]` called `stats` that will provide us with summary statistics about the values in the RDD. How can we get out summary stats of the `reading` column across the entire data set?

In [ ]:
print(data.map(lambda x: x.reading).stats())

How can we do the same but for only the position `FP1`?

In [ ]:
print(data.filter(lambda x: x.posn == "FP1") \
          .map(lambda x: x.reading).stats())

Let's make sure there are 256 samples per `posn` in this data set. How can we do this? 

In [ ]:
samples = data.map(lambda record: ((record.trial, record.posn), 1)) \
              .reduceByKey(lambda a, b: a + b)
posns = samples.map(lambda c: (c[0][1], c[1])).distinct()

print(posns.take(1))
print(posns.count(), posns.filter(lambda x: x[1] == 256).count())

## Joins in Spark


Like in MapReduce, a join is accomplished by emitting key-value pairs `(k, v)` and joining on similar keys.  It is accomplished by roughly the same mechanism (sending similar keys to the same *node*), although in Spark those key-value pairs may not (necessarily) be persisted to disk.

As an example, consider the case where we have two tables, stored here as CSV files.  One contains sales information, connecting users to particular transactions.

In [ ]:
! cat small_data/employee/sales.csv

We want some structure to represent each row of data in an RDD.  We could use a list or tuple, but in this case we'll make a simple class.

In [ ]:
class Transaction(object):
    def __init__(self, transactionId, productId, userId, amount):
        self.transactionId = transactionId
        self.productId = productId
        self.userId = userId
        self.amount = int(amount)
    
    @classmethod
    def from_line(cls, line):
        data = line.split(",")
        if data[0] == "sales":
            return cls(*data[1:])
        return None

transactions = sc.textFile(localpath("small_data/employee/sales.csv")) \
    .map(Transaction.from_line) \
    .filter(lambda x: x is not None)

The other table contains information about the users themselves.

In [ ]:
! cat small_data/employee/users.csv

In [ ]:
class User(object):
    def __init__(self, userId, email, language, country):
        self.userId = userId
        self.email = email
        self.language = language
        self.country = country
    
    @classmethod
    def from_line(cls, line):
        data = line.split(",")
        return cls(*data[1:])

users = sc.textFile(localpath("small_data/employee/users.csv")) \
    .map(User.from_line)

From the individual RDDs, we can make calculations about the total sales or distribution of users.

In [ ]:
totalSales = transactions.map(lambda x: x.amount).sum()
usersByCountryCount = users.map(lambda u: (u.country, u)).countByKey()

print("total sales %f" % totalSales)
print("Users by country")
for key, val in usersByCountryCount.items():
    print(key, val)

If we want to know the total sales per country, we need to join these RDDs.  To do this, we need a shared key, which in this case is the user ID.  We use map to transform both RDDs into key-value pairs, and then join to combine them into a single RDD.  This RDD has elements of the form `(userId, (transactionAmount, userCountry))`.  Another map and reduce by key is all we need to get the calculation we desire.

In [ ]:
salesByCountry = transactions.map(lambda t: (t.userId, t.amount)) \
                .join(users.map(lambda u: (u.userId, u.country))) \
                .map(lambda x: (x[1][1], x[1][0])) \
                .reduceByKey(lambda x, y: x + y) \
                .collect()

print("Transactions by country")
for country in salesByCountry:
    print(country[0], country[1])

### A warning about traversables


The transformations `mapPartitions`, `groupByKey`, `mapPartitionsWithIndex`, and `cogroup` return iterators.  Much like Python generators, you can only traverse them once.  Here's a common anti-pattern:

In [ ]:
# Anti-example

def mean(iterable):
    _size = sum(1 for _ in iterable)  # Not all iterables support len()
    _sum = float(sum(iterable))
    return _sum / _size

listMean = mean(range(1,6))
print("list mean: %f" % listMean)

iterMean = mean(iter(range(1, 6)))
print("iterator mean: %f" % iterMean)

### RDD Persistence

When you persist an RDD, each node stores any partitions of it that it computes in memory and reuses them in other actions on that data set (or data sets derived from it). This allows future actions to be much faster.

The argument passed to the persist method determines how the RDD will be stored. In general, you may choose
* Deserialized or serialized formats (serialized Java objects are more space-efficient but more CPU-intensive to read)
* In-memory only or allow spilling to disk
* *Fast* fault tolerance via redundancy
* [Full documentation](http://spark.apache.org/docs/latest/programming-guide.html#rdd-persistence)


*Serialization*: the process of translating data structures or object states into a format (i.e. series of bits) that can be stored and reconstructed later. Note that in Python, object storage is always handled by Pickle, so there's no distinction between serialized or deserialized.

1. The cache method is a shortcut for the default behavior: in memory and deserialized:
```python
myRdd.cache()
```
2. Otherwise, specify the desired behavior:
```python
myRdd.persist(MEMORY_ONLY_SER)
```
3. Cleaning up cached data is important for memory management. During shuffles, some intermediate data is automatically persisted, so it can be worthwhile to manually unpersist RDDs instead of waiting for garbage collection:
```python
myRDD.unpersist()
```
**Question:** Caching is a key tool for iterative algorithms and fast interactive use.  Why might you not always do it?

In [ ]:
def randomStream():
    return sc.parallelize(range(1,10000001)).map(lambda x: random.random())

uncachedLines = randomStream()
cachedLines = randomStream().cache()

In [ ]:
%timeit -n1 -r1 add = uncachedLines.reduce(lambda x, y: x + y)
%timeit -n1 -r1 sub = uncachedLines.reduce(lambda x, y: x - y)
%timeit -n1 -r1 mult = uncachedLines.reduce(lambda x, y: x * y)

In [ ]:
%timeit -n1 -r1 add = cachedLines.reduce(lambda x, y: x + y)
%timeit -n1 -r1 sub = cachedLines.reduce(lambda x, y: x - y)
%timeit -n1 -r1 mult = cachedLines.reduce(lambda x, y: x * y)

## Accumulators


We'll want to implement counters (like we had in `mrjob` and Hadoop in general).  As usual, here is a first anti-example:

In [ ]:
# Anti-example

class Counter(object):
    def __init__(self):
        self.counter = 0

    def increment(self, amt=1):
        self.counter += amt

counter = Counter()

sc.parallelize(range(1,11)).foreach(lambda x: counter.increment(1))

print(counter.counter)  # what's happening here?

Here's the correct way to implement this using [Accumulators](https://spark.apache.org/docs/1.3.0/api/scala/index.html#org.apache.spark.Accumulator).  We are just using them for integer addition but you can use them for any [monoid](https://en.wikipedia.org/wiki/Monoid).  You'll hear about these a lot in Scala.

In [ ]:
# Example

accumCounter = sc.accumulator(0)

sc.parallelize(range(1,11)).foreach(lambda x: accumCounter.add(1))

print(accumCounter)

In [ ]:
# Lazy example

def update(x):
    accumCounter.add(1)
    return x

accumCounter = sc.accumulator(0)

myRdd = sc.parallelize(range(1, 11)).map(update)

print("counter: ", accumCounter)

print(myRdd.reduce(lambda x, y: x + y))

print("counter: ", accumCounter)

myRdd.take(1)

print("counter: ", accumCounter)

## Broadcast variables


Broad variables are the Spark equivalent of Hadoop's distributed cache: they deposit read-only data cached on each machine rather than shipping a copy of it with tasks.

In [ ]:
# Basic usage:

iBV = sc.broadcast(10)
assert(iBV.value == 10)

Here's a more realistic example to show how you might implement this:

In [ ]:
class Sale(object):
    def __init__(self, state, sales):
        self.state = state
        self.sales = sales
        
def constructSale(tup):
    return Sale(tup[0], tup[1])

In [ ]:
states = {
    0: "AL",
    1: "KY",
    2: "VT",
}

statesBV = sc.broadcast(states)

data = sc.parallelize([(0, 30.), (1, 40.), (2, 20.), (2, 10.)]).map(constructSale)

In [ ]:
%%timeit -r1 -n100
# This works but it will send `states` to each node again if run twice
results1 = data.map(lambda s: (states[s.state], s.sales)).collect()

In [ ]:
%%timeit -r1 -n100
# broadcast to each node once, and can be used freely inside map
results2 = data.map(lambda s: (statesBV.value[s.state], s.sales)).collect()

In [ ]:
results1 = data.map(lambda s: (states[s.state], s.sales)).collect()
results2 = data.map(lambda s: (statesBV.value[s.state], s.sales)).collect()
assert(results1 == results2)

**Two caveats:**
1. After the broadcast variable is created, it should be used instead of the value so that it is not shipped to the nodes more than once.
2. The broadcast variable should not be modified after it is broadcast in order to ensure that all nodes get the same value of the broadcast variable.
3. How does the memory footprint of broadcasting scale with the number of tasks? With the number of executors?

**Question:** The above example is called a map-side join.  
1. Why is it called this?
1. What are the strengths and weaknesses of this versus a full join (i.e. emitting key-value pairs to common keys)?

In [ ]:
sc.stop()

*Copyright &copy; 2019 The Data Incubator.  All rights reserved.*